# Theoretical questions

The system (5) can be written as a problem on the form $Ax=b$, equivalently

$$
\begin{bmatrix}
D_{1,1} + \lambda A_{1,1} & D_{1,2} \\
D_{2,1} & D_{2,2} - \lambda A_{2,2}
\end{bmatrix}
\begin{bmatrix}
u \\
v
\end{bmatrix}
=
b.
$$

All matrices and vectors are of dimensions

$$
D_{1,1},\, D_{1,2},\, D_{2,1},\, D_{2,2},\, A_{1,1}, A_{2,2} \in \mathbb{R}^{(m n) \times (m n)}, \qquad
u,\, v \in \mathbb{R}^{m n}, \qquad
b \in \mathbb{R}^{2 m n}.
$$

$$
(A_{1,1})_{ij} = (A_{2,2})_{ij} =
\begin{cases}
4, & \text{if } i=j,\\
-1, & \text{if } |i-j|=1,\\
-1, & \text{if } |i-j|=m,\\
0, & \text{otherwise.}
\end{cases}
$$

$$
A_{1,1} = A_{2,2} = 
\begin{bmatrix}
4 & -1 &        &        & \cdots &        & -1     &        &        \\
-1& 4  & -1     &        &        & \cdots &        & -1     &        \\
   & -1 & 4     & -1     &        &        & \cdots &        & \ddots \\
   &    & \ddots& \ddots & \ddots &        &        &        &        \\
\vdots&  &       & -1     & 4     & -1     &        &        &        \\
      & \vdots  &        &        & -1     & 4      & -1     &        &        \\
-1    &         & \cdots &        &        & -1     & 4      & -1     &        \\
      & -1      &        & \cdots &        &        & -1     & 4      & -1     \\
      &         & \ddots &        &        &        &        & -1     & 4
\end{bmatrix}.
$$

And the diagonal matrices are

$$
(D_{1,1})_{i,j} =
\begin{cases}
0, & i \ne j, \\[6pt]
\displaystyle

\frac{1}{4} \Big( (I_0 - I_1)_{i+1,\,i} - (I_0 - I_1)_{i,\,i}
\Big)^{2},
& i = j.
\end{cases}
$$


$$
(D_{2,2})_{i,j} =
\begin{cases}
0, & i \ne j, \\[6pt]
\displaystyle

\frac{1}{4} \Big( (I_0 - I_1)_{i,\,i+1} - (I_0 - I_1)_{i,\,i}
\Big)^{2},
& i = j.
\end{cases}
$$

$$
(D_{1,2})_{i,j} = (D_{2,1})_{i,j} = 
\begin{cases}
0, & i \ne j, \\[6pt]
\displaystyle

\frac{1}{4} \Big( (I_0 - I_1)_{i,\,i+1} - (I_0 - I_1)_{i,\,i}
\Big) \Big( (I_0 - I_1)_{i+1,\,i} - (I_0 - I_1)_{i,\,i}
\Big),
& i = j.
\end{cases}
$$
